In [1]:
import pandas as pd
import random
import requests # 网页内容获取
import re # 解析数据
from lxml import etree # 解析数据
import time # 反反爬

In [2]:

session = requests.Session()  # 使用会话对象
def ua():
    """随机获取一个浏览器用户信息"""
    
    user_agents = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
    'Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7',
    'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0',
    ]

    return {'User-Agent': random.choice(user_agents)}


def get(url):
    """
    获取网页源码
    url: 目标网页的地址
    return:网页源码
    """
    res = requests.get(url=url, headers = ua())
    return res.text

def get_url(res_text):
    """
    获取源码中每个二手房详情页的url
    res_text:网页源码
    return:列表形式的30个二手房详情页的url
    """
    re_f = '<a class="" href="(.*?)" target="_blank"'
    url_list = re.findall(re_f, res_text)
    return url_list

In [3]:
url ='https://bj.lianjia.com/ershoufang/dongcheng/pg1/'
resp = get(url)
url_list = get_url(resp)
url_list

['https://bj.lianjia.com/ershoufang/101121960406.html',
 'https://bj.lianjia.com/ershoufang/101122041372.html',
 'https://bj.lianjia.com/ershoufang/101122021175.html',
 'https://bj.lianjia.com/ershoufang/101122021423.html',
 'https://bj.lianjia.com/ershoufang/101121940866.html',
 'https://bj.lianjia.com/ershoufang/101122008184.html',
 'https://bj.lianjia.com/ershoufang/101122003570.html',
 'https://bj.lianjia.com/ershoufang/101122031981.html',
 'https://bj.lianjia.com/ershoufang/101122020423.html',
 'https://bj.lianjia.com/ershoufang/101122021903.html',
 'https://bj.lianjia.com/ershoufang/101121750144.html',
 'https://bj.lianjia.com/ershoufang/101121771922.html',
 'https://bj.lianjia.com/ershoufang/101121818465.html',
 'https://bj.lianjia.com/ershoufang/101121749239.html',
 'https://bj.lianjia.com/ershoufang/101121894370.html',
 'https://bj.lianjia.com/ershoufang/101121708503.html',
 'https://bj.lianjia.com/ershoufang/101121873126.html',
 'https://bj.lianjia.com/ershoufang/101120574551

In [4]:
def get_else_data(res_text):
    
    res_text = etree.HTML(res_text)
    
    title = res_text.xpath("//div[@class='sellDetailHeader']//h1/@title")

    return dict(zip(['标题'], [title]))

def get_data(res_text):
    """获取房屋的详细数据"""
    res_text = etree.HTML(res_text)
    
    # 获取房屋的标题
    title = res_text.xpath("//div[@class='sellDetailHeader']//h1/@title")
    # 获取房屋的总价
    total_price = res_text.xpath("//div[@class='overview']//div/span/text()")[2]
    # 获取房屋的单价
    price = res_text.xpath("//div[@class='overview']//div/span/text()")[3]
    # 获取房屋的地段
    place = res_text.xpath("//div[@class='overview']//div/span/a/text()")
    
    ## 房屋基本信息获取
    # 获取房屋基本信息的标题
    lab = res_text.xpath("//div[@class='base']//span/text()")
    # 获取房屋基本信息的内容
    val = res_text.xpath("//div[@class='base']//li/text()")
    
    ## 获取房源交易信息
    # 获取房源交易标题
    key1 = res_text.xpath("//div[@class='transaction']//span[1]//text()")
    # 获取房源交易信息内容
    trans = res_text.xpath("//div[@class='transaction']//span[2]//text()")

    ## 获取房源特色信息
    # 获取房源特色标题
    key = res_text.xpath("//div[@class='baseattribute clear']/div[@class='name']/text()")
    # 获取房源特色内容
    val1 = res_text.xpath("//div[@class='baseattribute clear']/div[@class='content']/text()")
    
    # 返回包含上述信息的字典
    return dict(zip(['标题', '总价格', '单价', '地段'] + lab + key1 + key,
                    [title, total_price, price, place] + val + trans + val1))


In [5]:
url_list ="https://bj.lianjia.com/ershoufang/101121960406.html"
t_resp = get(url_list)
get_data(t_resp)

{'标题': ['中海紫御公馆采光好大阳台，南向两居'],
 '总价格': '1050',
 '单价': '116564',
 '地段': ['东城', '陶然亭'],
 '房屋户型': '2室2厅1厨1卫',
 '所在楼层': '低楼层 (共17层)',
 '建筑面积': '90.08㎡',
 '户型结构': '平层',
 '套内面积': '71.79㎡',
 '建筑类型': '板塔结合',
 '房屋朝向': '南',
 '建筑结构': '钢混结构',
 '装修情况': '精装',
 '梯户比例': '两梯六户',
 '供暖方式': '集中供暖',
 '配备电梯': '有',
 '挂牌时间': '2023-11-17',
 '交易权属': '商品房',
 '上次交易': '2012-03-26',
 '房屋用途': '普通住宅',
 '房屋年限': '满五年',
 '产权所属': '非共有',
 '抵押信息': '\n                                无抵押\n                              ',
 '房本备件': '已上传房本照片',
 '核心卖点': '\n                    低总价，房子虽然在三层，但是南向的采光没有大的影响\n                  ',
 '小区介绍': '\n                    中海紫御公馆是中海2010年在东城打造的高品质楼盘，小区楼间距大，外立面干挂石材现在看也和新房一样，人车分流，每个单元门口都有小喷泉，中海自持物业物业费3.28\n                  ',
 '户型介绍': '\n                    户型方正的南向两居，阳台面积比楼上的同户型都要大好多，两个南向的房间都很大，可以放大双人床和衣柜，明厨明卫\n                  ',
 '装修描述': '\n                    房子是2010年毛坯交房，新业主买过来需要重新装修了，户型方正，格局合理，采光充足\n                  '}

In [6]:

def main(qu, start_pg=1, end_pg=100, download_times=1, max_data_count=1100):
    for q in qu:
        url = 'https://bj.lianjia.com/ershoufang/' + q + '/'
        data = []
        filename = f'二手房-{q}第{download_times}次下载.csv'
        data_count = 0
        resume_pg = start_pg

        for i in range(start_pg, end_pg + 1):
            if data_count >= max_data_count:
                break

            new_url = url + f'pg{i}/'
            a_new_url = get(new_url)
            url_list = get_url(a_new_url)
            time.sleep(random.uniform(1, 2))  # 调整等待时间

            for url_item in url_list:
                if data_count >= max_data_count:
                    break

                text = get(url_item)
                data.append(get_data(text))
                data_count += 1

            resume_pg = i
            time.sleep(random.uniform(2, 4))  # 调整等待时间

            if i % 5 == 0:
                pd.DataFrame(data).to_csv(filename, encoding='GB18030')
                print(f'前{i}页数据已保存')

        with open(f'resume_{q}.txt', 'w') as f:
            f.write(str(resume_pg))


In [7]:

main(['dongcheng', 'haidian', 'tongzhou', 'huairou'])

KeyboardInterrupt: 